This notebook cleans .txt and .dta data for students performance on exams for BIO1111 and exports a csv which can be analyzed in an R script.
#### Author: Christopher Agard

In [1]:
import pandas as pd
import numpy as np
import os,glob
from soar import *

pd.options.display.max_columns = 50

# Setting source and output folders

In [4]:
tu={'worksource': "C:/Users/tuh27554/Documents/Grading/2020/SPR EX1/Input Data",
    'workoutput': "C:/Users/tuh27554/Documents/Grading/2020/SPR EX1/Output Data"}
gandolf={'worksource': "C:/Users/craga/Google Drive/TU 2020/SPR/Exam 1/Input data",
    'workoutput': "C:/Users/craga/Google Drive/TU 2020/SPR/Exam 1/Output Data"}

source = tu

# Getting and Cleaning Exam Data

In [5]:
os.chdir(source['worksource'])
files = glob.glob('*.dta')
files

['marcexams_2019_2019-A2132.dta']

In [7]:
os.chdir(source['worksource'])
df1=cleanSoarExam(files[0],examNum=1,soarSessions=[73,74,75])
df1['version'] = 1
# df2=cleanSoarExam(files[1],examNum=1,soarSessions=[73,74,75])
# df2['version'] = 2
df = df1#.append(df2)
df.head()

C:\Users\tuh27554\Documents\GitHub\Grading\soar.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  tmp['soar'] = soar


,tuid,last,first,middle,unnamed1,unnamed2,unnamed3,soar,ncorrect,examNumber,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,soarType,version
0,500 - BLU,EXAM - KEY,NNNNNN,N,NaN,NaN,NaN,NaN,27,1,1,3,2,3,4,2,4,1,2,4,1,3,2,2,3,1,4,3,4,3,1,2,1,3,2,1,1,key,1
1,915388683,ROGERS,COREY,A,11.0,NaN,NaN,72.0,11,1,4,3,2,3,4,4,4,3,3,4,4,3,2,4,1,1,3,3,1,4,4,1,0,4,1,1,4,other,1
2,915786206,POLLACK,JASON,H,11.0,NaN,NaN,75.0,19,1,3,2,2,3,4,2,4,1,2,4,1,3,4,4,3,1,3,2,4,3,1,4,1,2,2,1,1,mine,1
3,915694573,SHEKHTERMAN,HELEN,NaN,11.0,NaN,NaN,NaN,15,1,4,1,2,3,4,2,4,2,2,4,1,1,4,2,3,2,1,3,4,4,4,4,1,3,1,1,3,other,1
4,915597565,BETSILL,ANDREA,C,11.0,12.0,79.0,78.0,21,1,1,3,2,3,4,2,4,1,2,2,1,3,2,4,1,1,4,3,4,2,4,2,1,3,2,1,3,other,1


We will confirm that the exam key have been appropriately identified and change it if necessary.

Now we change the dir to the outputfolder and save the cleaned, merged file.

In [8]:
itemcols = df.columns[df.columns.str.contains('item')]
itemcols

Index(['item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6', 'item_7',
       'item_8', 'item_9', 'item_10', 'item_11', 'item_12', 'item_13',
       'item_14', 'item_15', 'item_16', 'item_17', 'item_18', 'item_19',
       'item_20', 'item_21', 'item_22', 'item_23', 'item_24', 'item_25',
       'item_26', 'item_27'],
      dtype='object')

In [9]:
for col in itemcols:
    df[col] = df[col].apply(letters2num)

In [12]:
os.chdir(source['workoutput'])
df.to_csv("SpringExam 1_cleaned.csv")

# Analyzing Exam Data

1. [Most Frequently Wrong](#Most-Frequently-Wrong-Answers)

## Strange values for items

In reviewing the data I noted some values outside of the expected 1-4.  Could you help me understand what these are?

In [13]:
all_responses = pd.concat([pd.Series(df[col].unique()) for col in itemcols]).unique()
strange_response = [resp for resp in all_responses if resp not in ['A','B','C','D','E','1','2','3','4','5']]
strange_resp_item=[col for col in itemcols if df.loc[df[col].isin(strange_response),col].shape[0]>0]
print("The strange responses {} were found for the following items:\n{}".format(strange_response, strange_resp_item))


The strange responses [' ', 0, nan] were found for the following items:
['item_7', 'item_10', 'item_11', 'item_19', 'item_20', 'item_21', 'item_23', 'item_27']


## Most Frequently Wrong Answers

Here we identify the columns containing information about responses.

In [14]:
df.soarType.value_counts()

other    258
mine      73
key       18
Name: soarType, dtype: int64

Now we apply this function to create a ranked list of items by order of the number of my students getting them wrong.

## Sec 75

In [18]:
wrongcount_v1_mine = pd.DataFrame()
for col in itemcols:
    wrongcount_v1_mine = wrongcount_v1_mine.append(countwrong(df= df.loc[df.soar==75], col = col, ver = 1,compind=['mine','other']))
wrongcount_v1_mine = wrongcount_v1_mine.sort_values(['number','item'],ascending=False).reset_index(drop = True)
# wrongcount_v1_mine.index = wrongcount_v1_mine.item
# wrongcount_v1_mine= wrongcount_v1_mine.drop(columns = 'item')
wrongcount_v1_mine

,item,number
0,item_21,16
1,item_27,14
2,item_1,14
3,item_22,13
4,item_8,12
5,item_10,12
6,item_20,11
7,item_13,11
8,item_19,10
9,item_11,10


In [20]:
total_wrong75 = wrongcount_v1_mine.set_index(['item']).sum(axis=1)
total_wrong75.to_csv('total wrong 75.csv')
total_wrong75.sort_values(ascending=False)

item
item_21    16
item_27    14
item_1     14
item_22    13
item_8     12
item_10    12
item_20    11
item_13    11
item_19    10
item_11    10
item_9      9
item_14     9
item_5      8
item_17     8
item_16     8
item_25     7
item_15     6
item_6      5
item_12     5
item_4      4
item_26     4
item_24     4
item_23     4
item_2      4
item_18     2
item_7      0
item_3      0
dtype: int64

## Printing the Keys

In [22]:
# import numpy as np
v1keys = df.loc[(df.soar.isnull())&(df.soarType=='key')&(df.version==1),itemcols].iloc[0]
# v2keys = df.loc[(df.soarType=='key')&(df.version==2),itemcols].iloc[0]
keys = pd.DataFrame(data = {'version1':v1keys})#,'version2':v2keys})
keys.to_csv('1111 exam 1 S2020 keys.csv',index = False)
keys

,version1
item_1,D
item_2,B
item_3,C
item_4,B
item_5,A
item_6,C
item_7,A
item_8,D
item_9,C
item_10,A


# Evaluating Student Performance

## Score distribution
Now we look at scores by section. 

In [28]:
df.columns

Index(['tuid', 'last', 'first', 'middle', 'unnamed1', 'unnamed2', 'unnamed3',
       'soar', 'ncorrect', 'examNumber', 'item_1', 'item_2', 'item_3',
       'item_4', 'item_5', 'item_6', 'item_7', 'item_8', 'item_9', 'item_10',
       'item_11', 'item_12', 'item_13', 'item_14', 'item_15', 'item_16',
       'item_17', 'item_18', 'item_19', 'item_20', 'item_21', 'item_22',
       'item_23', 'item_24', 'item_25', 'item_26', 'item_27', 'soarType',
       'version'],
      dtype='object')

In [36]:
dfDesc = df.loc[(df.soarType!='key')].groupby(['soarType','soar']).ncorrect.describe()
dfDesc.loc[dfDesc['count']>1]

count       mean       std   min    25%   50%    75%   max
soarType soar                                                            
mine     73.0   31.0  17.806452  4.085905  10.0  14.50  19.0  20.50  24.0
         74.0   21.0  17.952381  3.721239  11.0  15.00  18.0  21.00  24.0
         75.0   21.0  17.000000  3.619392  10.0  15.00  18.0  19.00  22.0
other    71.0   34.0  16.676471  4.087828   9.0  14.25  17.5  19.75  23.0
         72.0   28.0  17.035714  3.327574  10.0  15.75  17.5  19.00  23.0
         77.0   29.0  17.068966  3.807240  10.0  14.00  16.0  20.00  24.0
         78.0   27.0  16.777778  4.236593   8.0  13.50  17.0  20.00  23.0
         80.0   30.0  17.466667  3.655259   9.0  15.25  17.0  20.75  24.0
         81.0   15.0  16.600000  3.112188  10.0  15.50  17.0  19.00  21.0
         82.0   28.0  17.321429  3.752072  12.0  14.00  17.0  20.25  24.0
         85.0   26.0  17.269231  3.768901   9.0  15.25  17.0  20.50  23.0
         86.0   17.0  18.705882  4.011931  10.0  18.00  20.0  22.00  23.0

In [24]:
df.groupby(['soar']).ncorrect.describe()

,count,mean,std,min,25%,50%,75%,max
soar,,,,,,,,
1.0,2.0,23.500000,4.949747,20.0,21.75,23.5,25.25,27.0
2.0,2.0,22.500000,6.363961,18.0,20.25,22.5,24.75,27.0
4.0,2.0,23.500000,4.949747,20.0,21.75,23.5,25.25,27.0
58.0,2.0,23.500000,4.949747,20.0,21.75,23.5,25.25,27.0
71.0,35.0,16.971429,4.389062,9.0,14.50,18.0,20.00,27.0
72.0,29.0,17.379310,3.755128,10.0,16.00,18.0,19.00,27.0
73.0,32.0,18.093750,4.335594,10.0,14.75,19.0,21.00,27.0
74.0,22.0,18.363636,4.112067,11.0,15.25,18.0,21.75,27.0
75.0,22.0,17.454545,4.125730,10.0,15.00,18.5,19.75,27.0


## Item Analysis
1. [Percent choice for each item](#Percent-choice-for-each-item)

os.chdir(source['worksource'])
df20 = pd.read_csv('Spring 2020v1.csv')
dfcomp = pd.read_csv('Cross-Walk 2019 Fall to 2018 Fall Exams.csv')
dfcomp

### Percent choice for each item

df_item1 = pd.DataFrame()
for col in itemcols:
    df_item1 = df_item1.append(df.loc[df.version==1,col].value_counts(normalize=True))
df_item1 = df_item1.applymap(lambda x: "{:.0f}".format(x*100))
df_item1 = df_item1.reset_index().rename(columns={'index':'F19E1'})
df_item1['F19E1'] = df_item1.F19E1.apply(lambda x: int(x[5:]))
df_item1 = df_item1.sort_values('F19E1')
df_item1['F18E1'] = dfcomp.F18E1
df_item1.to_csv('Fall 2019v1.csv')
df_item1

df_item = pd.DataFrame()
for col in itemcols:
    df_item = df_item.append(df.loc[df.version==2,col].value_counts(normalize=True))
df_item = df_item.applymap(lambda x: "{:.0f}".format(x*100))
df_item

df18

dfcomp